# Parse data from .txt or .csv files

In [53]:
def parse_txt(filename, oriented=True):
    """
    Parse data from txt file into dict python type.
    JSON serializable.
    """
    data = {}
    with open(filename) as file:
        
        line = file.readline()
        while line:
            
            # skip comments
            if line[0] == '#':
                line = file.readline()
                continue
            
            parent, child = line.split()
            parent = int(parent)
            child = int(child)
            
            # rows in data file can be duplicated
            if parent in data:
                if child not in data[parent]['linked']:
                    data[parent]['linked'].append(child)
                    data[parent]['degree'] += 1
            else:
                data[parent] = { 
                    'linked': [child],
                    'distances': {},
                    'degree': 1,
                    'centrality': 0,
                    'marked': False,
                    'active': True
                }
                
            if not oriented:
                if child in data:
                    if parent not in data[child]['linked']:
                        data[child]['linked'].append(parent)
                        data[child]['degree'] += 1

                else:    
                    data[child] = {
                        'linked': [parent],
                        'distances': {},
                        'degree': 1,
                        'centrality': 0,
                        'marked': False,
                        'active': True
                    }

            line = file.readline()

    return data

def parse(filename, oriented=True):
    if filename.split('.')[-1] == 'txt':
        return parse_txt(filename, oriented)

# Distance counting methods with modifications

In [54]:
from collections import deque 
import datetime as dt

def count_distance(vertice, data, h = -1, full=False, rollback=True, monitoring=False):
    """
    Counts distances form given vertice to all other in connectivity component that vertice belongs to.
    Also, of h parameter is provided, this method finds list of vertices which are h or less away from provided vertice.
    (As only distance from provided vertive becomes more then h method stops.)
    Based on BFS.
    vertice: index of source vertice
    data: dict with information about graph
    h: distance to closest vertices
    fill: complete BFS in spite of current distance > h
    """

    operations_counter = 0
    timestamp_before_algorithm = dt.datetime.now()
    
    current_distance = 0
    centrality = 0
    vertices_number = 1
    nearest_vertices = []
    d0 = deque()
    d1 = deque()
    
    source_active = data[vertice]['active']
    
    d0.append(vertice)
    data[vertice]['marked'] = True
    
    operations_counter += 9
    
    while True:
        
        if (not d0 and not d1) or (h != -1 and current_distance > h and not full):
            operations_counter += 9
            break
        
        if current_distance % 2 == 0:
            
            v = d0.pop()
                
            for i in data[v]['linked']:
                if not data[i]['marked']:
                    d1.append(i)
                    data[i]['marked'] = True
                    
                    operations_counter += 2
                operations_counter += 2
            
            data[v]['distances'][vertice] = current_distance
            vertices_number += 1
            centrality += current_distance
            
            # set active to false if it vertice 'v' is too close to source vertice 'vertice'
            if h != -1 and current_distance <= h:
                data[v]['active'] = False
                nearest_vertices.append(v)
                
                operations_counter += 2
                
            # go to the next level of distance from the source vertice 'vertice'
            if not d0:
                current_distance += 1
                                
                operations_counter += 1
                
            operations_counter += 5 # number of "if"'s
                
        else:
            
            v = d1.pop()
                
            for i in data[v]['linked']:
                if not data[i]['marked']:
                    d0.append(i)
                    data[i]['marked'] = True
                    
                    operations_counter += 2
                operations_counter += 2
            
            data[v]['distances'][vertice] = current_distance
            vertices_number += 1
            centrality += current_distance
            
            # set active to false if it vertice 'v' is too close to source vertice 'vertice'
            if h != -1 and current_distance <= h:
                data[v]['active'] = False
                nearest_vertices.append(v)
                
                operations_counter += 3
            
            # go to the next level of distance from the source vertice 'vertice'
            if not d1:
                current_distance += 1
                
                operations_counter += 1
                
            operations_counter += 5 # number of "if"'s added
    
    # rollback data
    if rollback:
        for key, value in data.items():
            value['marked'] = False
    
    # set initial status
    data[vertice]['active'] = source_active
    
    operations_counter =+ 1
    
    if full:
        data[vertice]['centrality'] = centrality / vertices_number
        operations_counter += 4
        
        if h == -1:
            if monitoring:
                return operations_counter, dt.datetime.now() - timestamp_before_algorithm
        else:
            if monitoring:
                return nearest_vertices, operations_counter, dt.datetime.now() - timestamp_before_algorithm
            return nearest_vertices
    else:
        if h == -1:
            if monitoring:
                return operations_counter, dt.datetime.now() - timestamp_before_algorithm
        else:
            if monitoring:
                return nearest_vertices, operations_counter, dt.datetime.now() - timestamp_before_algorithm
            return nearest_vertices
    
        

def bfs(source, stock, data, rollback=True, monitoring=False):
    """
    Count distance from source to stock without using landmarks.
    Pure BFS.
    """
    operations_counter = 0
    timestamp_before_algorithm = dt.datetime.now()
    
    if not all(key in data for key in (source, stock)):
        operations_counter += 2 * len(data.items())
        if monitoring:
            return -1, operations_counter, dt.datetime.now() - timestamp_before_algorithm
        return -1
    
    current_distance = 0
    flag_found = False
    d0 = deque()
    d1 = deque()
    
    d0.append(source)
    data[source]['marked'] = True
    
    operations_counter += 7
    
    while True:
        
        if not d0 and not d1:
            current_distance = -1
            
            operations_counter += 3
            break
        
        if current_distance % 2 == 0:
            
            v = d0.pop()
            
            if v == stock:
                flag_found = True
                
                operations_counter += 1
                break
                
            for i in data[v]['linked']: 
                if not data[i]['marked']:
                    d1.append(i)
                    data[i]['marked'] = True
                    
                    operations_counter += 2
                operations_counter += 2
            
            if not d0:
                current_distance += 1
                
                operations_counter += 1
                
            operations_counter += 3
                
        else:
            
            v = d1.pop()
            
            if v == stock:
                flag_found = True
                break
                
            for i in data[v]['linked']:
                if not data[i]['marked']:
                    d0.append(i)
                    data[i]['marked'] = True
                    
                    operations_counter += 2
                operations_counter += 2
            
            if not d1:
                current_distance += 1 
                
                operations_counter += 1
            
            operations_counter += 3
    
    if rollback:
        for key, value in data.items():
            value['marked'] = False

    if flag_found:
        if monitoring:
            return current_distance, operations_counter, dt.datetime.now() - timestamp_before_algorithm
        return current_distance
    else:
        if monitoring:
            return -1, operations_counter, dt.datetime.now() - timestamp_before_algorithm
        return -1

# Select landmarks using random, degree or centrality ranking

In [55]:
import random
from math import log
import datetime as dt

def select_landmarks(
    data: dict,
    number_of_landmarks = 0.1,
    ranking: str = 'degree',
    h: int = 1,
    rollback=False,
    monitoring=False
):
    """
    Select landmarks using constratined strategy with provided h and ranking parameters.
    Set h to -1 to get top 'number_of_landmarks' ranked vertices.
    Possible ranking: degree, random, closeness.
    Set rollback to True to roll back all vertices 'active' to True
    """
    operations_counter = 0
    timestamp_before_algorithm = dt.datetime.now()
    
    data_items = data.items()
    graph_size = len(data_items)
#     print('=' * 126)
#     print('Graph size: ' + str(graph_size))
    
    number_of_landmarks = int(
        graph_size * (number_of_landmarks / 100)
    ) if number_of_landmarks >= 1 else int(
        graph_size * number_of_landmarks
    )
    
#     print('=' * 126)
#     print('Target number of landmarks: ' + str(number_of_landmarks))
    
    landmarks = []
    
    operations_counter += graph_size + 5
    
    
    if ranking == 'random':
        
        vertices = [i[0] for i in data_items]
        if h == -1:
            
            random_vertices = [vertices[i] for i in random.sample(range(0,graph_size), number_of_landmarks)]
            
            for v in vertices:
                operations, __ = count_distance(vertice=v, data=data, h=-1, full=True, monitoring=True)
                operations_counter += operations + 1
            
            operations_counter += 1 + number_of_landmarks
            
            if monitoring:
                return random_vertices, operations_counter, dt.datetime.now() - timestamp_before_algorithm
            return random_vertices

        operations_counter += len(data_items) + number_of_landmarks
        
        random.shuffle(vertices)
        
        operations_counter += len(vertices)
        
        while len(landmarks) < number_of_landmarks and vertices:
            
            v = vertices.pop(0)
            
            # check if vertice 'v' is less than 'h' away from some previously selected landmark
            if not data[v]['active']:
                continue
            
            landmarks.append(v)
            _, operations, __ = count_distance(vertice=v, data=data, h=h, full=True, monitoring=True)
            
            operations_counter += operations + 3
    
    
    elif ranking == 'degree':
        
        data_sorted = sorted(data_items, key=lambda x: x[1]['degree'], reverse=True)
        
        operations_counter += graph_size * log(graph_size)
        
        if h == -1:
            
            max_degree_vertices = [i[0] for i in data_sorted[:number_of_landmarks]]
            
            for v in max_degree_vertices:
                operations, __ = count_distance(vertice=v, data=data, h=-1, full=True, monitoring=True)
                operations_counter += operations + 1
            
            operations_counter += 2 * number_of_landmarks + 1
            if monitoring:
                return max_degree_vertices, operations_counter, dt.datetime.now() - timestamp_before_algorithm
            return max_degree_vertices
        
        while len(landmarks) < number_of_landmarks and data_sorted:
            v = data_sorted.pop(0)[0]
            
            # check if vertice 'v' is less than 'h' away from some previously selected landmark
            if not data[v]['active']:
                continue
                
            landmarks.append(v)
            _, operations, __ = count_distance(vertice=v, data=data, h=h, full=True, monitoring=True)
            
            operations_counter += operations + 3
            
    
    elif ranking == 'closeness':
        
        vertices = [i[0] for i in data_items]

        number_of_seeds = int(
            number_of_landmarks + (graph_size - number_of_landmarks) / 2
        ) if number_of_landmarks >= (graph_size / 2) else number_of_landmarks * 2
#         print('=' * 126)
#         print('Number of seeds: ' + str(number_of_seeds))
        
        random_indexes = random.sample(range(0,graph_size), number_of_seeds)
        seeds = [vertices[i] for i in random_indexes]
        
        operations_counter += number_of_seeds + len(random_indexes) + 6
        
        if h == -1:
            
            for v in seeds:
                operations, __ = count_distance(vertice=v, data=data, h=-1, full=True, monitoring=True)
                operations_counter += number_of_landmarks + operations
                
            # select top 'number_of_landmarks' vertices by centrality
            max_centrality_vertices = [
                i[0] for i in sorted(
                    [(key, value['centrality']) for key, value in data.items() if key in seeds],
                    key=lambda x: x[1],
                    reverse=True
                )[:number_of_landmarks]
            ]
            
            operations_counter += 1 + 2 * len(seeds) + log(len(seeds)) * len(seeds)
            if monitoring:
                return max_centrality_vertices, operations_counter, dt.datetime.now() - timestamp_before_algorithm
            return max_centrality_vertices
        
            
        for seed in seeds:
            _, operations, __ = count_distance(vertice=seed, data=data, h=h, full=True, monitoring=True)
            operations_counter += operations

        seeds_sorted = [
            i[0] for i in sorted(
                [(key, value['centrality']) for key, value in data.items() if key in seeds],
                key= lambda x: x[1],
                reverse=True
            )
        ]
        
        landmarks.append(seeds_sorted.pop(0))
#         print(str(landmarks[0]) + ': ' + str(data[landmarks[0]]['active']))

        operations_counter += graph_size + len(seeds) * log(len(seeds))

        while len(landmarks) < number_of_landmarks:
            
            if not seeds_sorted:
#                 print('=' * 126)
#                 print('For unsortet vertices: ')
#                 print("Number of landmarks after iteration by sorted seeds: " + str(len(landmarks)))
#                 print('=' * 126)
                for key, value in data.items():
                    if len(landmarks) >= number_of_landmarks:
                        break
                    if value['active'] and key not in landmarks:
                        _, operations, __ = count_distance(vertice=key, data=data, h=h, full=True, monitoring=True)
                        landmarks.append(key)
                        operations_counter += operations + 1
                    
                    operations_counter += 2
                break
            
            v = seeds_sorted.pop(0)
#             print(str(v) + ': ' + str(data[v]['active']))
            # check if vertice 'v' is less than 'h' away from some previously selected landmark
            if not data[v]['active']:
                continue
            
            landmarks.append(v) 
#         print('=' * 126)
       
            operations_counter += 3
        
            
    # data rollback
    if rollback:
        for key, value in data.items():
            value['active'] = True
                
    if monitoring:
        landmarks, operations_counter, dt.datetime.now() - timestamp_before_algorithm
    return landmarks

# Shortest path estimating via landmarks method

In [56]:
def shortest_path(
    source,
    stock,
    landmarks,
    data,
    estimation_strategy='geometrical_mean',
    monitoring=False
):
    """
    Counts distance from source to stock using landmarks.
    For distance estimation geometric mean is used.
    There are 4 estimation strategies: geometrical mean, upper, lower and middle point. Geometrical mean is default.
    """
    
    operations_counter = 0
    timestamp_before_algorithm = dt.datetime.now()
    
    # source or stock not in graph
    try:
        source_distances = data[source]['distances']
        stock_distances = data[stock]['distances']
    except KeyError:
        if monitoring:
            return -1, operations_counter, dt.datetime.now() - timestamp_before_algorithm
        return -1
    
    L = -1
    U = 3 * graph_size
    
    operations_counter += 5
    
    for key, value in source_distances.items():
        
        # use distances only to landmarks
        if key not in landmarks:
            continue
        
        temp = stock_distances.get(key, -1)
        # cheks if there are distance from stock to 'key' landmark
        if temp == -1:
            continue
        
        l = abs(value - temp)
        u = value + temp
        
        if l > L:
            L = l
            operations_counter += 1
        if u < U:
            U = u
            operations_counter += 1
            
        operations_counter += 9 + len(landmarks)
            
    if L == -1 and U == 3 * graph_size:
        # this mean that source and stock are in different connectivity components
        
        operations_counter += 3
        
        if monitoring:
            return -1, operations_counter, dt.datetime.now() - timestamp_before_algorithm
        return -1
    
    # choose estimating strategy
    if estimation_strategy == 'geometrical_mean':
        result = (L * U) ** 0.5
    elif estimation_strategy == 'upper':
        result = U
    elif estimation_strategy == 'lower':
        result = L
    elif estimation_strategy == 'middle_point':
        result = (L + U) / 2
    else:
        result = (L * U) ** 0.5
    
    if monitoring:
        return result, operations_counter, dt.datetime.now() - timestamp_before_algorithm
    return result

In [96]:
data = parse('../datasets/CA-AstroPh.txt', oriented=False)
landmarks = select_landmarks(data, 0.05, 'degree')

print('=' * 126)
print('Number of calculated landmarks: ' + str(len(landmarks)))
print('=' * 126)
print('Landmarks: ')
print([{'vertice_id':key, 'active':value['active'], 'degree':value['degree'], 'centrality': value['centrality']} for key, value in data.items() if key in landmarks])
print('=' * 126)
for landmark in landmarks:
    assert data[landmark]['active'] != False

Graph size: 18772
Target number of landmarks: 938
Number of calculated landmarks: 938
Landmarks: 
[{'vertice_id': 276, 'active': True, 'degree': 31, 'centrality': 0}, {'vertice_id': 16442, 'active': True, 'degree': 30, 'centrality': 0}, {'vertice_id': 29829, 'active': True, 'degree': 13, 'centrality': 0}, {'vertice_id': 64124, 'active': True, 'degree': 27, 'centrality': 0}, {'vertice_id': 77915, 'active': True, 'degree': 8, 'centrality': 0}, {'vertice_id': 89308, 'active': True, 'degree': 32, 'centrality': 0}, {'vertice_id': 90506, 'active': True, 'degree': 49, 'centrality': 0}, {'vertice_id': 95070, 'active': True, 'degree': 9, 'centrality': 0}, {'vertice_id': 124023, 'active': True, 'degree': 30, 'centrality': 0}, {'vertice_id': 132445, 'active': True, 'degree': 132, 'centrality': 0}, {'vertice_id': 3365, 'active': True, 'degree': 103, 'centrality': 0}, {'vertice_id': 8445, 'active': True, 'degree': 27, 'centrality': 0}, {'vertice_id': 42512, 'active': True, 'degree': 9, 'centrality'

# Environment variables

In [50]:
FILENAME = 'test.txt'
ORIENTED = False
LANDMARKS_PERCENT = 10
TEST_VERTICES_PERCENT = 10

# Test results

In [57]:
data = parse(FILENAME, ORIENTED)
graph_size = len(data.items())

select_strategies = ('random', 'degree', 'centrality')
estimation_strategies = ('geometrical_mean', 'middle_point', 'upper', 'lower')

test_vertices_number = int(
    graph_size * (TEST_VERTICES_PERCENT / 100)
) if TEST_VERTICES_PERCENT > 1 else int(
    graph_size * TEST_VERTICES_PERCENT
)

results = {
    'random': {
        'geometrical_mean': [],
        'middle_point': [],
        'upper': [],
        'lower': []
    },
    'degree': {
        'geometrical_mean': [],
        'middle_point': [],
        'upper': [],
        'lower': []
    },
    'centrality': {
        'geometrical_mean': [],
        'middle_point': [],
        'upper': [],
        'lower': []
    }
}

vertices = [i[0] for i in data.items()]
for select in select_strategies:
    for estimation in estimation_strategies:
        
        print('=' * 126)
        print(select)
        print(estimation)
        
        landmarks = select_landmarks(data, LANDMARKS_PERCENT, select, monitoring=True, rollback=True)

        vertices_source = vertices.copy()
        random.shuffle(vertices_source)
        vertices_source = vertices_source[:test_vertices_number]
        
        vertices_stock = vertices.copy()
        random.shuffle(vertices_stock)
        vertices_stock = vertices_stock[:test_vertices_number]
        
        for source, stock in zip(vertices_source,vertices_stock):
            
            print('=' * 126)
            print('Source: ' + str(source))
            print('Stock: ' + str(stock))

            distance_bfs, operations_bfs, time_bfs = bfs(source, stock, data, monitoring=True)
            
            distance_landmarks, operations_landmarks, time_landmarks = shortest_path(source, stock, landmarks, data, monitoring=True)
            
            accuracy = abs(distance_bfs - distance_landmarks)
            operation_delta = abs(operations_bfs - operations_landmarks)
            time_delta = abs(time_bfs - time_landmarks)
            
            print('Accuracy: ' + str(accuracy))
            print('Operations delta: ' + str(operation_delta))
            print('Operating time delta: ' + str(time_delta))
            
            results[select][estimation].append((accuracy, operation_delta, time_delta))

random
geometrical_mean
Source: 53056
Stock: 3365
Accuracy: 1.1715728752538097
Operations delta: 1221
Operating time delta: 0:00:00
Source: 129273
Stock: 2969
Accuracy: 0.7084973778708186
Operations delta: 1921
Operating time delta: 0:00:00
Source: 76749
Stock: 50897
Accuracy: 1.1715728752538097
Operations delta: 737
Operating time delta: 0:00:00.001000
Source: 6058
Stock: 7026
Accuracy: 1.1715728752538097
Operations delta: 1748
Operating time delta: 0:00:00
Source: 3365
Stock: 8583
Accuracy: 0.0
Operations delta: 1190
Operating time delta: 0:00:00.001000
Source: 61686
Stock: 113335
Accuracy: 1.127016653792583
Operations delta: 1833
Operating time delta: 0:00:00
Source: 109056
Stock: 20188
Accuracy: 0.7084973778708186
Operations delta: 1906
Operating time delta: 0:00:00.001001
Source: 106274
Stock: 76854
Accuracy: 0.7320508075688772
Operations delta: 1090
Operating time delta: 0:00:00
Source: 131234
Stock: 75607
Accuracy: 0.8377223398316205
Operations delta: 496
Operating time delta: 0

Source: 39298
Stock: 122003
Accuracy: 1.127016653792583
Operations delta: 1806
Operating time delta: 0:00:00
Source: 127393
Stock: 122425
Accuracy: 0.0
Operations delta: 1140
Operating time delta: 0:00:00.001000
Source: 20188
Stock: 10639
Accuracy: 0.0
Operations delta: 445
Operating time delta: 0:00:00
Source: 106954
Stock: 67410
Accuracy: 1.127016653792583
Operations delta: 1501
Operating time delta: 0:00:00
Source: 53681
Stock: 115700
Accuracy: 0.0
Operations delta: 603
Operating time delta: 0:00:00.001000
Source: 65713
Stock: 38109
Accuracy: 1.127016653792583
Operations delta: 1661
Operating time delta: 0:00:00
Source: 8445
Stock: 119042
Accuracy: 1.1715728752538097
Operations delta: 472
Operating time delta: 0:00:00
Source: 99504
Stock: 42972
Accuracy: 2.5358983848622456
Operations delta: 1911
Operating time delta: 0:00:00.001001
Source: 11050
Stock: 39298
Accuracy: 1.1715728752538097
Operations delta: 1489
Operating time delta: 0:00:00
Source: 77098
Stock: 125678
Accuracy: 0.0
Op

Operations delta: 2696
Operating time delta: 0:00:00
Source: 70950
Stock: 131592
Accuracy: 3
Operations delta: 750
Operating time delta: 0:00:00
Source: 115420
Stock: 11580
Accuracy: 5
Operations delta: 2197
Operating time delta: 0:00:00.001000
Source: 111660
Stock: 121988
Accuracy: 7
Operations delta: 4057
Operating time delta: 0:00:00
Source: 97012
Stock: 55999
Accuracy: 3
Operations delta: 1381
Operating time delta: 0:00:00
Source: 51588
Stock: 107829
Accuracy: 5
Operations delta: 2821
Operating time delta: 0:00:00
Source: 77915
Stock: 27281
Accuracy: 6
Operations delta: 4125
Operating time delta: 0:00:00
Source: 42994
Stock: 60310
Accuracy: 4
Operations delta: 1633
Operating time delta: 0:00:00
Source: 61461
Stock: 115209
Accuracy: 5
Operations delta: 2744
Operating time delta: 0:00:00
Source: 131234
Stock: 29829
Accuracy: 5
Operations delta: 3048
Operating time delta: 0:00:00.001000
Source: 53056
Stock: 97101
Accuracy: 5
Operations delta: 2841
Operating time delta: 0:00:00
Source:

In [59]:
for key, value in data.items():
    value['marked'] = False
distance_bfs = bfs(123233, 44011, data)
distance_landmarks = shortest_path(123233, 44011, landmarks, data)
print(distance_bfs)
print(distance_landmarks)

4
-1


In [36]:
print(results)

[(3138, datetime.timedelta(0)), (1099, datetime.timedelta(0)), (1585, datetime.timedelta(0)), (2642, datetime.timedelta(0)), (2892, datetime.timedelta(microseconds=1001)), (1472, datetime.timedelta(0)), (2649, datetime.timedelta(0)), (2662, datetime.timedelta(0)), (2624, datetime.timedelta(microseconds=1000)), (1105, datetime.timedelta(0)), (2734, datetime.timedelta(0)), (1676, datetime.timedelta(0)), (4095, datetime.timedelta(microseconds=1000)), (2498, datetime.timedelta(0)), (1002, datetime.timedelta(0)), (1505, datetime.timedelta(0)), (2672, datetime.timedelta(0)), (3809, datetime.timedelta(microseconds=1000)), (1569, datetime.timedelta(0)), (3063, datetime.timedelta(0)), (3266, datetime.timedelta(0)), (1586, datetime.timedelta(0)), (2839, datetime.timedelta(0)), (2944, datetime.timedelta(0)), (2905, datetime.timedelta(0)), (1236, datetime.timedelta(microseconds=1000)), (2643, datetime.timedelta(0)), (765, datetime.timedelta(0)), (446, datetime.timedelta(0)), (1872, datetime.timede

In [ ]:
d = {
    1: {
        'linked': [2,3,4,10],
        'distances':{},
        'marked': False
    },
    2: {
        'linked': [1,5,6,7],
        'distances':{},
        'marked': False
    },
    3: {
        'linked': [1,8],
        'distances':{},
        'marked': False
    },
    4: {
        'linked': [1,9,10],
        'distances':{},
        'marked': False
    },
    5: {
        'linked': [2],
        'distances':{},
        'marked': False
    },
    6: {
        'linked': [2],
        'distances':{},
        'marked': False
    },
    7: {
        'linked': [2],
        'distances':{},
        'marked': False
    },
    8: {
        'linked': [3],
        'distances':{},
        'marked': False
    },
    9: {
        'linked': [4],
        'distances':{},
        'marked': False
    },
    10: {
        'linked': [4,1],
        'distances':{},
        'marked': False
    }
}

print(bfs(1, 3, d))

In [16]:
def a():
    return [1,2,3], 4,5
x,y,z = a()
print(x)

[1, 2, 3]
